In [10]:
import numpy as np
import cv2
import open3d as o3d
import time

# 读取深度图像
depth_img = cv2.imread("../img/1_depth.png", cv2.IMREAD_GRAYSCALE)


In [11]:
w, h = depth_img.shape
# 相机内参矩阵
k = np.array([[535.4, 0.0, 320.0],
              [0.0, 539.2, 240.0],
              [0.0, 0.0, 1.0]])



def depth_to_point_cloud_v2(depth):
    # (w*h,3)，实际绝大多数时间用在这里了
    static_array = np.array([np.array([i,j, 1]) for i in range(w) for j in range(h)])
    
    # 获取相机内参的逆矩阵
    k_inv = np.linalg.inv(k)

    # 每行乘逆矩阵
    point_array = k_inv@static_array.T

    depth_flat = depth.astype(np.float32)
    depth_flat = depth_flat.reshape(-1,1)
    # 每行乘深度值
    point_cloud = depth_flat * point_array.T
    return point_cloud

pc = depth_to_point_cloud_v2(depth_img)

In [16]:
# 十次求平均
elapsed_time_list = []
for i in range(10):# 
    start_time = time.time()  # 记录开始时间

    # 将深度图转换为点云
    pc = depth_to_point_cloud_v2(depth_img)

    end_time = time.time()  # 记录结束时间

    elapsed_time = end_time - start_time  # 计算耗时

    elapsed_time_list.append(elapsed_time)

average_time = sum(elapsed_time_list) / len(elapsed_time_list)

print(f"Time elapsed average 10 times:{average_time:.4f} seconds.")

Time elapsed average 10 times:0.0044 seconds.


In [13]:
import cProfile
cProfile.run('depth_to_point_cloud_v2(depth_img)')


         307226 function calls in 0.278 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.068    0.068    0.193    0.193 2593389271.py:11(<listcomp>)
        1    0.022    0.022    0.277    0.277 2593389271.py:9(depth_to_point_cloud_v2)
        1    0.001    0.001    0.278    0.278 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 linalg.py:130(get_linalg_error_extobj)
        1    0.000    0.000    0.000    0.000 linalg.py:135(_makearray)
        2    0.000    0.000    0.000    0.000 linalg.py:140(isComplexType)
        1    0.000    0.000    0.000    0.000 linalg.py:153(_realType)
        1    0.000    0.000    0.000    0.000 linalg.py:159(_commonType)
        1    0.000    0.000    0.000    0.000 linalg.py:203(_assert_stacked_2d)
        1    0.000    0.000    0.000    0.000 linalg.py:209(_assert_stacked_square)
        1    0.000    0.000    0.000    0.000 linalg.py:488(_unary_dispatcher)
 

In [14]:
# 可视化点云

# 点云数据存储在名为point_cloud的numpy数组中
cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(pc)

# 创建一个可视化窗口
vis = o3d.visualization.Visualizer()
vis.create_window()

# 将点云添加到可视化窗口中
vis.add_geometry(cloud)

# 设置相机位置，没调所以可有可无
vis.get_view_control().set_front([0, 0, -1])
vis.get_view_control().set_lookat([0, 0, 0])
vis.get_view_control().set_up([0, -1, 0])

# 开始显示点云
vis.run()

# 关闭显示窗口
vis.destroy_window()

0.2s